In [ ]:
import psycopg2
import json

def connect_to_database():
    dbname = 'ozqervrg'
    user = 'ozqervrg'  # Your username is typically the same as the dbname
    password = 'Z4gq6SqdsnoeV28T6h1gJ-YsMsT9JxTr'
    host = 'kashin.db.elephantsql.com'
    port = '5432'  # Default port for PostgreSQL

    try:
        # Establish the connection
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )

        # Create a cursor object
        cur = conn.cursor()

        # SQL command to create the users table
        create_table_query = '''
            CREATE TABLE IF NOT EXISTS usersdoc (
                id bigint PRIMARY KEY,
                id_str varchar(255),
                name varchar(255),
                screen_name varchar(255),
                location varchar(255),
                url varchar(255),
                description text,
                verified boolean,
                followers_count integer,
                friends_count integer,
                listed_count integer,
                favourites_count integer,
                statuses_count integer
            )
        '''

        # Execute the SQL command
        cur.execute(create_table_query)

        # Commit the transaction
        conn.commit()

        # If table creation is successful, print a success message
        print("Table 'usersdoc' created successfully!")

        # Return the connection and cursor objects
        return conn, cur

    except psycopg2.OperationalError as e:
        # If connection fails, print the error message
        print(f"Unable to connect to PostgreSQL database: {e}")
        return None, None

def insert_user(conn, cur, userdoc, limit=20000):
    """Insert a new user into the database."""
    insert_stmt = """
    INSERT INTO usersdoc (id, id_str, name, screen_name, location, url, description, verified, 
    followers_count, friends_count, listed_count, favourites_count, statuses_count)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (id) DO NOTHING;
    """
    cur.execute(insert_stmt, (
        userdoc['id'], 
        userdoc['id_str'], 
        userdoc['name'], 
        userdoc['screen_name'],
        userdoc.get('location', ''), 
        userdoc.get('url', ''), 
        userdoc.get('description', ''),
        userdoc['verified'], 
        userdoc['followers_count'], 
        userdoc['friends_count'],
        userdoc['listed_count'], 
        userdoc['favourites_count'], 
        userdoc['statuses_count']
    ))
    conn.commit()

def load_data(source):
    # Connect to the database
    conn, cur = connect_to_database()
    if conn and cur:
        try:
            with open(source, 'r') as file:
                for line_number, line in enumerate(file, start=1):
                    try:
                        tweet_data = json.loads(line)
                        # Extract the user information from the tweet data
                        user_data = tweet_data.get('user', None)
                        if user_data:
                            insert_user(conn, cur, user_data)
                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON data at line {line_number}: {e}")
                        print(f"Line content: {line}")
                        continue  # Skip to the next line if there's an error
        except FileNotFoundError:
            print(f"Error: File '{source}' not found.")
        finally:
            # Close the cursor and connection
            cur.close()
            conn.close()

# Call the load_data function
load_data("corona-out-3")



